In [4]:
import ast
import re
import json
#import spacy
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as px
import numpy as np

%matplotlib inline
plt.rcParams['font.family'] = 'Times New Roman'

In [7]:
persona_char = pd.read_csv('../Data/final_persona_char_movie_role.csv')
persona_char.columns

Index(['Wikipedia movie ID', 'character', 'pagerank', 'character_type',
       'full_name', 'freebase_char_map', 'wikipedia_movie_id', 'movie_name',
       'related_name', 'token', 'persona_code', 'persona_distribution',
       'freebase_movie_id', 'character_name', 'actor_dob', 'actor_gender',
       'actor_height', 'actor_ethnicity', 'actor_name', 'actor_age_release',
       'freebase_char_id', 'freebase_actor_id', 'movie_release_date',
       'movie_box_office_revenue', 'movie_runtime', 'movie_genres_cleaned',
       'movie_countries_cleaned', 'movie_languages_cleaned',
       'movie_release_year'],
      dtype='object')

In [10]:
# groupby by persona ids and get the proportion of each persona
persona_char_counts = persona_char.groupby(['persona_code']).size().reset_index(name='counts')
persona_char_counts.to_csv('../Data/persona/persona_char_counts.csv', index=False)

In [2]:
persona_word = json.load(open('../Data/Persona/persona_word.json', 'r'))
categories = list(persona_word.keys())

all_words = []
for category in categories:
    all_words += persona_word[category]["top_verbs"]

all_words = list(set(all_words))

# Create a matrix where rows correspond to categories and columns to words
word_matrix = np.zeros((len(categories), len(all_words)))
for i, category in enumerate(categories):
    for word in persona_word[category]["top_verbs"]:
        word_matrix[i][all_words.index(word)] = persona_word[category]["top_freq"][persona_word[category]["top_verbs"].index(word)]

word_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(word_matrix)

x = pca_result[:,0]
y = pca_result[:,1]

f = go.FigureWidget([go.Scatter(x=x, y=y, mode='markers')])
scatter = f.data[0]
colors = ['#a3a7e4'] * 100
scatter.marker.color = colors
scatter.marker.size = [10] * 100
f.layout.hovermode = 'closest'


# create our callback function
def update_point(trace, points, selector):
    c = list(scatter.marker.color)
    s = list(scatter.marker.size)
    for i in points.point_inds:
        c[i] = '#bae2be'
        s[i] = 20
        with f.batch_update():
            scatter.marker.color = c
            scatter.marker.size = s


scatter.on_click(update_point)

display(f)

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                                   #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4, #a3a7e4,
                         

In [5]:
final_merged = pd.read_csv('../Data/final_merged_df.csv')
final_merged.columns

Index(['Wikipedia movie ID', 'char A', 'action', 'char B', 'combined_action',
       'lemmatized_action', 'char A type', 'char B type', 'freebase_movie_id',
       'movie_name', 'movie_release_date', 'movie_box_office_revenue',
       'movie_runtime', 'movie_languages', 'movie_countries', 'movie_genres',
       'year', 'character_name_charA', 'actor_dob_charA', 'actor_gender_charA',
       'actor_height_charA', 'actor_ethnicity_charA', 'actor_name_charA',
       'actor_age_charA', 'freebase_character_map_1_charA',
       'freebase_character_map_2_charA', 'freebase_character_map_3_charA',
       'character_name_charB', 'actor_dob_charB', 'actor_gender_charB',
       'actor_height_charB', 'actor_ethnicity_charB', 'actor_name_charB',
       'actor_age_charB', 'freebase_character_map_1_charB',
       'freebase_character_map_2_charB', 'freebase_character_map_3_charB',
       'crime', 'violence', 'sadness', 'anger', 'love', 'fear', 'care',
       'grief', 'mentor', 'confrontation', 'heartbre

In [2]:
# get the persona distribution of males
male_data = final_merged[final_merged['actor_gender'] == 'M']
female_data = final_merged[final_merged['actor_gender'] == 'M']


In [ ]:
fig = px.Figure(data=[px.Scatter(
	x=random_x,
	y=random_y,
	mode='markers',)
])
 
fig.update_layout(
	updatemenus=[
		dict(
		buttons=list([
			dict(
			args=["type", "scatter"],
			label="Scatter Plot",
			method="restyle"
			),
                        dict(
                        args=["type", "box"],
                        label="Box Plot",
                        method="restyle"
                        ),
			dict(
			args=["type", "bar"],
			label="Bar Plot",
			method="restyle"
			)
			]),
			direction="down",
		    ),
	      ],
      title = "Dropdown Menu in plotly Graph Using Python"
)
 
fig.show()